In [1]:
import pandas
import numpy as np
from pyiron_base import Settings
from pyiron_base.archiving.import_archive import update_id_lst
from pyiron_base.archiving.export_archive import new_job_id
from pyiron_atomistics import Project
s = Settings()


# Copy definition of export_database function, but comment unnecessary lines
def export_database_as_df(
    pr
    #project_instance, directory_to_transfer,
    #archive_directory
):
    # here we first check wether the archive directory is a path
    # or a project object
    #if isinstance(archive_directory, str):
    #    archive_directory = os.path.basename(archive_directory)
    # if the archive_directory is a project
    #elif static_isinstance(
    #    obj=archive_directory.__class__,
    #    obj_type=[
    #        "pyiron_base.project.generic.Project",
    #    ]
    #):
    #    archive_directory = archive_directory.path
    #else:
     #   raise RuntimeError(
     #       """the given path for exporting to,
     #       does not have the correct format paths as string
     #       or pyiron Project objects are expected"""
     #   )
    #directory_to_transfer = os.path.basename(directory_to_transfer)
    #pr = project_instance.open(os.curdir)
    df = pr.job_table()
    job_ids_sorted = sorted(df.id.values)
    new_job_ids = list(range(len(job_ids_sorted)))
    job_translate_dict = {
        j: n for j, n in zip(job_ids_sorted, new_job_ids)
    }
    df['id'] = [
        new_job_id(
            job_id=job_id, job_translate_dict=job_translate_dict
        )
        for job_id in df.id
    ]
    df['masterid'] = [
        new_job_id(
            job_id=job_id, job_translate_dict=job_translate_dict
        )
        for job_id in df.masterid
    ]
    df['parentid'] = [
        new_job_id(job_id=job_id, job_translate_dict=job_translate_dict)
        for job_id in df.parentid
    ]
    #df['project'] = update_project(
    #    project_instance,
    #    directory_to_transfer=directory_to_transfer,
    #    archive_directory=archive_directory, df=df
    #)
    #del df["projectpath"]
    return df

# Taken from import_jobs, but without the unnecessary lines
def import_dataframe_to_db(pr, df):
    job_id_lst = []
    for entry in df.dropna(axis=1).to_dict(orient="records"):
        if 'id' in entry:
            del entry['id']
        if 'parentid' in entry:
            del entry['parentid']
        if 'masterid' in entry:
            del entry['masterid']
        if 'timestart' in entry:
            entry["timestart"] = pandas.to_datetime(entry["timestart"])
        if 'timestop' in entry:
            entry["timestop"] = pandas.to_datetime(entry["timestop"])
        if 'username' not in entry: 
            entry["username"] = s.login_user
        job_id = pr.db.add_item_dict(par_dict=entry)
        job_id_lst.append(job_id)
    
    # Update parent and master ids
    for job_id, masterid, parentid in zip(
        job_id_lst,
        update_id_lst(record_lst=df["masterid"].values, job_id_lst=job_id_lst),
        update_id_lst(record_lst=df["parentid"].values, job_id_lst=job_id_lst),
    ):
        if not np.isnan(masterid) or not np.isnan(parentid):
            pr.db.item_update(
                item_id=job_id,
                par_dict={
                    "parentid": parentid, "masterid": masterid
                }
            )

In [2]:
s._configuration['sql_connection_string'] = "sqlite:///db1.db"

In [3]:
pr = Project("ProjectDB1")
pr.remove_jobs_silently()
ref = pr.create_job("Lammps", "Ref")
ref.structure = pr.create_ase_bulk("Cu", cubic=True)
ref.potential = ref.list_potentials()[0]

murn = ref.create_job("Murnaghan", "murn")
murn.run()
df_1 = export_database_as_df(pr)
df_1

The job murn was saved and received the ID: 1
The job strain_0_9 was saved and received the ID: 2


/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas

The job strain_0_92 was saved and received the ID: 3


/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "


The job strain_0_94 was saved and received the ID: 4


...

/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "


The job strain_1_1 was saved and received the ID: 12
job_id:  2 finished


/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "


job_id:  3 finished
job_id:  4 finished
job_id:  5 finished
job_id:  6 finished
job_id:  7 finished
job_id:  8 finished
job_id:  9 finished
job_id:  10 finished
job_id:  11 finished
job_id:  12 finished


,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,0,finished,Cu4,murn,/murn,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/,2021-06-11 14:05:30.297538,2021-06-11 14:05:35.851804,5.0,pyiron_test@PC#1#11/11,Murnaghan,0.3.0,None,NaN
1,1,finished,Cu4,strain_0_9,/strain_0_9,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:30.643882,2021-06-11 14:05:30.800220,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
2,2,finished,Cu4,strain_0_92,/strain_0_92,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:31.121597,2021-06-11 14:05:31.278069,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
3,3,finished,Cu4,strain_0_94,/strain_0_94,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:31.600593,2021-06-11 14:05:31.749007,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
4,4,finished,Cu4,strain_0_96,/strain_0_96,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:32.076108,2021-06-11 14:05:32.226670,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
5,5,finished,Cu4,strain_0_98,/strain_0_98,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:32.548196,2021-06-11 14:05:32.705140,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
6,6,finished,Cu4,strain_1_0,/strain_1_0,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:33.039370,2021-06-11 14:05:33.196301,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
7,7,finished,Cu4,strain_1_02,/strain_1_02,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:33.545215,2021-06-11 14:05:33.690813,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
8,8,finished,Cu4,strain_1_04,/strain_1_04,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:34.023840,2021-06-11 14:05:34.172408,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
9,9,finished,Cu4,strain_1_06,/strain_1_06,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:34.492665,2021-06-11 14:05:34.640453,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0


In [4]:
s.open_local_sqlite_connection("sqlite:///db2.db")

In [5]:
pr2 = Project("ProjectDB2")
pr2.remove_jobs_silently()
ref = pr2.create_job("Lammps", "Ref")
ref.structure = pr2.create_ase_bulk("Al", cubic=True)
ref.potential = ref.list_potentials()[0]

murn = ref.create_job("Murnaghan", "murn")
murn.run()
df_2 = export_database_as_df(pr2)
df_2

The job murn was saved and received the ID: 1
The job strain_0_9 was saved and received the ID: 2


/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas

The job strain_0_92 was saved and received the ID: 3


/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "


The job strain_0_94 was saved and received the ID: 4


...

/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "


The job strain_1_1 was saved and received the ID: 12


/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity indices. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity velocities. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "
/home/niklas/git_projects/pyiron_atomistics/pyiron_atomistics/lammps/units.py:201: UserWarning: Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity steps. Returning un-normalized quantity
  warnings.warn(message="Warning: Couldn't determine the LAMMPS to pyiron unit conversion type of quantity "


job_id:  2 finished
job_id:  3 finished
job_id:  4 finished
job_id:  5 finished
job_id:  6 finished
job_id:  7 finished
job_id:  8 finished
job_id:  9 finished
job_id:  10 finished
job_id:  11 finished
job_id:  12 finished


,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,0,finished,Al4,murn,/murn,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/,2021-06-11 14:05:36.872687,2021-06-11 14:05:43.035660,6.0,pyiron_test@PC#1#11/11,Murnaghan,0.3.0,None,NaN
1,1,finished,Al4,strain_0_9,/strain_0_9,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:37.232790,2021-06-11 14:05:37.396585,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
2,2,finished,Al4,strain_0_92,/strain_0_92,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:37.757920,2021-06-11 14:05:37.918036,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
3,3,finished,Al4,strain_0_94,/strain_0_94,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:38.278969,2021-06-11 14:05:38.429737,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
4,4,finished,Al4,strain_0_96,/strain_0_96,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:38.787357,2021-06-11 14:05:38.939806,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
5,5,finished,Al4,strain_0_98,/strain_0_98,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:39.298959,2021-06-11 14:05:39.454319,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
6,6,finished,Al4,strain_1_0,/strain_1_0,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:39.817246,2021-06-11 14:05:39.985694,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
7,7,finished,Al4,strain_1_02,/strain_1_02,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:40.357155,2021-06-11 14:05:40.517492,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
8,8,finished,Al4,strain_1_04,/strain_1_04,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:40.900405,2021-06-11 14:05:41.064105,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0
9,9,finished,Al4,strain_1_06,/strain_1_06,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:41.449830,2021-06-11 14:05:41.619588,0.0,pyiron_test@PC#1,Lammps,0.1,None,0.0


In [6]:
s.open_local_sqlite_connection("sqlite:///db3.db")
pr3 = Project("ProjectDB3")
pr3.db

In [7]:
import_dataframe_to_db(df=df_1, pr=pr3)

In [8]:
pr3 = Project(".")
pr3.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,Cu4,murn,/murn,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/,2021-06-11 12:30:50.812518,2021-06-11 12:30:56.246277,5.0,pyiron_test@PC#1#11/11,Murnaghan,0.3.0,None,NaN
1,2,finished,Cu4,strain_0_9,/strain_0_9,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:51.153598,2021-06-11 12:30:51.305323,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
2,3,finished,Cu4,strain_0_92,/strain_0_92,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:51.622300,2021-06-11 12:30:51.779604,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
3,4,finished,Cu4,strain_0_94,/strain_0_94,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:52.098094,2021-06-11 12:30:52.242662,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
4,5,finished,Cu4,strain_0_96,/strain_0_96,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:52.560811,2021-06-11 12:30:52.721982,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,finished,Cu4,strain_1_02,/strain_1_02,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:33.545215,2021-06-11 14:05:33.690813,0.0,pyiron_test@PC#1,Lammps,0.1,None,85.0
92,93,finished,Cu4,strain_1_04,/strain_1_04,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:34.023840,2021-06-11 14:05:34.172408,0.0,pyiron_test@PC#1,Lammps,0.1,None,85.0
93,94,finished,Cu4,strain_1_06,/strain_1_06,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:34.492665,2021-06-11 14:05:34.640453,0.0,pyiron_test@PC#1,Lammps,0.1,None,85.0
94,95,finished,Cu4,strain_1_08,/strain_1_08,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 14:05:34.956991,2021-06-11 14:05:35.111736,0.0,pyiron_test@PC#1,Lammps,0.1,None,85.0


In [9]:
import_dataframe_to_db(df=df_2, pr=pr3)
pr3.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,Cu4,murn,/murn,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/,2021-06-11 12:30:50.812518,2021-06-11 12:30:56.246277,5.0,pyiron_test@PC#1#11/11,Murnaghan,0.3.0,None,NaN
1,2,finished,Cu4,strain_0_9,/strain_0_9,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:51.153598,2021-06-11 12:30:51.305323,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
2,3,finished,Cu4,strain_0_92,/strain_0_92,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:51.622300,2021-06-11 12:30:51.779604,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
3,4,finished,Cu4,strain_0_94,/strain_0_94,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:52.098094,2021-06-11 12:30:52.242662,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
4,5,finished,Cu4,strain_0_96,/strain_0_96,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB1/murn_hdf5/,2021-06-11 12:30:52.560811,2021-06-11 12:30:52.721982,0.0,pyiron_test@PC#1,Lammps,0.1,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,104,finished,Al4,strain_1_02,/strain_1_02,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:40.357155,2021-06-11 14:05:40.517492,0.0,pyiron_test@PC#1,Lammps,0.1,None,97.0
104,105,finished,Al4,strain_1_04,/strain_1_04,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:40.900405,2021-06-11 14:05:41.064105,0.0,pyiron_test@PC#1,Lammps,0.1,None,97.0
105,106,finished,Al4,strain_1_06,/strain_1_06,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:41.449830,2021-06-11 14:05:41.619588,0.0,pyiron_test@PC#1,Lammps,0.1,None,97.0
106,107,finished,Al4,strain_1_08,/strain_1_08,/home/niklas/git_projects/TransferJobsToOtherDB/,ProjectDB2/murn_hdf5/,2021-06-11 14:05:42.000319,2021-06-11 14:05:42.162433,0.0,pyiron_test@PC#1,Lammps,0.1,None,97.0
